In [1]:
import tensorflow as tf
from TPM_from_VCS.data import toy_data_generator
from TPM_from_VCS.models.etp import etp
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

##########################
### WRAPPER FUNCTIONS
##########################

def fully_connected(input_tensor, output_nodes,
                    activation=None, seed=None,
                    name='fully_connected'):

    with tf.name_scope(name):
        input_nodes = input_tensor.get_shape().as_list()[1]
        weights = tf.Variable(tf.truncated_normal(shape=(input_nodes,
                                                         output_nodes),
                                                  mean=0.0,
                                                  stddev=0.1,
                                                  dtype=tf.float32,
                                                  seed=seed),
                              name='weights')
        biases = tf.Variable(tf.zeros(shape=[output_nodes]), name='biases')

        act = tf.matmul(input_tensor, weights) + biases
        if activation is not None:
            act = activation(act)
        return act


##########################
### DATASET
##########################

mnist = input_data.read_data_sets("./", validation_size=0)


##########################
### SETTINGS
##########################


# Hyperparameters
learning_rate = 0.01
training_epochs = 5000
batch_size = 128

# Architecture
hidden_size = 2
input_size = 5
# image_width = 28
MAX_VC_DIM = 10
MAX_NODES = 1000

# Other
print_interval = 20
random_seed = 123


##########################
### GRAPH DEFINITION
##########################

g = tf.Graph()
with g.as_default():
    
    tf.set_random_seed(random_seed)
    
#     cost_history = []

    # Input data
    input_layer = tf.placeholder(tf.float32, [None, input_size],
                                 name='input')

    ###########
    # Encoder
    ###########
    
    hidden_layer = fully_connected(input_layer, hidden_size, 
                                   activation=None, 
                                   name='encoding')
    
    ###########
    # Decoder
    ###########
    
    logits = fully_connected(hidden_layer, input_size, 
                             activation=None, name='logits')
    # note MNIST pixels are normalized to 0-1 range
#     out_layer = tf.nn.sigmoid(logits, name='decoding') 
    
    # set the output layer to be the same as logits. TODO: try relu activation also. Change the cost funcion definition only
    out_layer = tf.nn.relu(logits, name='decoding') 
    
    ##################
    # Loss & Optimizer
    ##################
    
    cost = tf.reduce_mean(tf.losses.mean_squared_error(
        labels=input_layer, predictions=logits), name='cost')
    
#     cost_history.append(cost)
    
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(cost, name='train')

    # Saver to save session for reuse
    saver = tf.train.Saver()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
batch_x = toy_data_generator.create_dataset(1)

batch_x[0][0]

len(batch_x)

61

In [ ]:
import numpy as np

##########################
### TRAINING & EVALUATION
##########################
    
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    cost_history = []
    
    np.random.seed(random_seed) # random seed for mnist iterator
    for epoch in range(training_epochs):
        avg_cost = 0.
#         total_batch = mnist.train.num_examples // batch_size
        total_batch = 500
        batch_x = toy_data_generator.create_dataset(1)
        
        for i in range(total_batch):
            
            for z in range(len(batch_x)):
                _, c = sess.run(['train', 'cost:0'], 
                                feed_dict={'input:0': np.reshape(batch_x[z][0], (-1, 5))})
                avg_cost += c
                
            cost_history.append(avg_cost)
                
            
            if not i % print_interval:
                print("Minibatch: %03d | Cost:    %.3f" % (i + 1, c))
                
    
        print("Epoch:     %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)))
    
    saver.save(sess, save_path='./autoencoder.ckpt')

Minibatch: 001 | Cost:    133923.281
Minibatch: 021 | Cost:    68463.109
Minibatch: 041 | Cost:    71358.953
Minibatch: 061 | Cost:    73208.203
Minibatch: 081 | Cost:    74011.289
Minibatch: 101 | Cost:    74903.047
Minibatch: 121 | Cost:    75496.414
Minibatch: 141 | Cost:    75850.406
Minibatch: 161 | Cost:    78346.273
Minibatch: 181 | Cost:    75951.734
Minibatch: 201 | Cost:    76023.883
Minibatch: 221 | Cost:    76092.336
Minibatch: 241 | Cost:    76134.305
Minibatch: 261 | Cost:    76224.844
Minibatch: 281 | Cost:    76313.719
Minibatch: 301 | Cost:    76396.625
Minibatch: 321 | Cost:    75482.516
Minibatch: 341 | Cost:    75772.078
Minibatch: 361 | Cost:    76311.062
Minibatch: 381 | Cost:    76238.789
Minibatch: 401 | Cost:    76351.984
Minibatch: 421 | Cost:    78054.273
Minibatch: 441 | Cost:    76165.953
Minibatch: 461 | Cost:    76359.594
Minibatch: 481 | Cost:    76135.602
Epoch:     001 | AvgCost: 3050009.511
Minibatch: 001 | Cost:    5714.844
Minibatch: 021 | Cost:    

Minibatch: 461 | Cost:    151482.906
Minibatch: 481 | Cost:    153991.406
Epoch:     009 | AvgCost: 2131088.532
Minibatch: 001 | Cost:    1405.207
Minibatch: 021 | Cost:    1970.393
Minibatch: 041 | Cost:    2191.834
Minibatch: 061 | Cost:    2440.882
Minibatch: 081 | Cost:    2172.328
Minibatch: 101 | Cost:    2006.008
Minibatch: 121 | Cost:    3273.043
Minibatch: 141 | Cost:    1999.382
Minibatch: 161 | Cost:    2931.062
Minibatch: 181 | Cost:    1975.539
Minibatch: 201 | Cost:    3399.565
Minibatch: 221 | Cost:    1961.498
Minibatch: 241 | Cost:    2166.824
Minibatch: 261 | Cost:    1929.750
Minibatch: 281 | Cost:    5272.122
Minibatch: 301 | Cost:    1982.137
Minibatch: 321 | Cost:    1935.362
Minibatch: 341 | Cost:    7708.444
Minibatch: 361 | Cost:    1922.711
Minibatch: 381 | Cost:    1982.300
Minibatch: 401 | Cost:    1905.269
Minibatch: 421 | Cost:    1942.448
Minibatch: 441 | Cost:    2535.527
Minibatch: 461 | Cost:    6139.091
Minibatch: 481 | Cost:    2051.140
Epoch:     01

Minibatch: 341 | Cost:    24761.781
Minibatch: 361 | Cost:    34197.879
Minibatch: 381 | Cost:    24209.178
Minibatch: 401 | Cost:    24608.992
Minibatch: 421 | Cost:    24721.645
Minibatch: 441 | Cost:    24757.869
Minibatch: 461 | Cost:    24771.291
Minibatch: 481 | Cost:    24777.354
Epoch:     018 | AvgCost: 350083.354
Minibatch: 001 | Cost:    57438.168
Minibatch: 021 | Cost:    13230.232
Minibatch: 041 | Cost:    17437.789
Minibatch: 061 | Cost:    18210.447
Minibatch: 081 | Cost:    18331.143
Minibatch: 101 | Cost:    18277.043
Minibatch: 121 | Cost:    18192.295
Minibatch: 141 | Cost:    18106.035
Minibatch: 161 | Cost:    18024.182
Minibatch: 181 | Cost:    17948.434
Minibatch: 201 | Cost:    17879.965
Minibatch: 221 | Cost:    17820.176
Minibatch: 241 | Cost:    17771.496
Minibatch: 261 | Cost:    17731.621
Minibatch: 281 | Cost:    17651.887
Minibatch: 301 | Cost:    17810.326
Minibatch: 321 | Cost:    17883.760
Minibatch: 341 | Cost:    17892.270
Minibatch: 361 | Cost:    1

Minibatch: 221 | Cost:    90221.195
Minibatch: 241 | Cost:    89925.078
Minibatch: 261 | Cost:    89868.250
Minibatch: 281 | Cost:    89713.727
Minibatch: 301 | Cost:    89774.539
Minibatch: 321 | Cost:    89794.336
Minibatch: 341 | Cost:    89556.078
Minibatch: 361 | Cost:    89478.891
Minibatch: 381 | Cost:    89487.875
Minibatch: 401 | Cost:    89361.391
Minibatch: 421 | Cost:    89409.094
Minibatch: 441 | Cost:    89208.273
Minibatch: 461 | Cost:    89113.953
Minibatch: 481 | Cost:    89324.359
Epoch:     027 | AvgCost: 5275992.847
Minibatch: 001 | Cost:    95931.422
Minibatch: 021 | Cost:    81876.719
Minibatch: 041 | Cost:    81347.602
Minibatch: 061 | Cost:    81492.758
Minibatch: 081 | Cost:    80951.625
Minibatch: 101 | Cost:    81057.344
Minibatch: 121 | Cost:    81419.172
Minibatch: 141 | Cost:    80989.680
Minibatch: 161 | Cost:    80724.336
Minibatch: 181 | Cost:    80723.586
Minibatch: 201 | Cost:    81027.359
Minibatch: 221 | Cost:    80904.758
Minibatch: 241 | Cost:    

Minibatch: 101 | Cost:    2829.148
Minibatch: 121 | Cost:    2829.090
Minibatch: 141 | Cost:    2894.718
Minibatch: 161 | Cost:    3404.959
Minibatch: 181 | Cost:    2927.393
Minibatch: 201 | Cost:    2856.333
Minibatch: 221 | Cost:    2882.361
Minibatch: 241 | Cost:    2855.135
Minibatch: 261 | Cost:    2857.974
Minibatch: 281 | Cost:    3007.513
Minibatch: 301 | Cost:    6955.104
Minibatch: 321 | Cost:    6528.103
Minibatch: 341 | Cost:    2986.735
Minibatch: 361 | Cost:    3018.315
Minibatch: 381 | Cost:    2802.961
Minibatch: 401 | Cost:    2865.631
Minibatch: 421 | Cost:    2775.120
Minibatch: 441 | Cost:    3738.410
Minibatch: 461 | Cost:    2894.125
Minibatch: 481 | Cost:    2861.476
Epoch:     036 | AvgCost: 5401996.406
Minibatch: 001 | Cost:    178871.531
Minibatch: 021 | Cost:    14328.252
Minibatch: 041 | Cost:    6804.224
Minibatch: 061 | Cost:    4964.068
Minibatch: 081 | Cost:    3807.375
Minibatch: 101 | Cost:    3792.729
Minibatch: 121 | Cost:    3125.416
Minibatch: 141

...

Minibatch: 301 | Cost:    112731.539
Minibatch: 321 | Cost:    112604.016
Minibatch: 341 | Cost:    112265.688
Minibatch: 361 | Cost:    112354.711
Minibatch: 381 | Cost:    112662.000
Minibatch: 401 | Cost:    112274.375
Minibatch: 421 | Cost:    112371.852
Minibatch: 441 | Cost:    112396.836
Minibatch: 461 | Cost:    112098.461
Minibatch: 481 | Cost:    112305.641
Epoch:     862 | AvgCost: 5040508.836
Minibatch: 001 | Cost:    8332.363
Minibatch: 021 | Cost:    773.580
Minibatch: 041 | Cost:    827.671
Minibatch: 061 | Cost:    2237.479
Minibatch: 081 | Cost:    930.540
Minibatch: 101 | Cost:    990.139
Minibatch: 121 | Cost:    986.654
Minibatch: 141 | Cost:    981.806
Minibatch: 161 | Cost:    976.897
Minibatch: 181 | Cost:    964.258
Minibatch: 201 | Cost:    932.412
Minibatch: 221 | Cost:    765.932
Minibatch: 241 | Cost:    694.506
Minibatch: 261 | Cost:    769.877
Minibatch: 281 | Cost:    784.070
Minibatch: 301 | Cost:    1107.935
Minibatch: 321 | Cost:    1532.223
Minibatch:

Minibatch: 241 | Cost:    134253.203
Minibatch: 261 | Cost:    133978.312
Minibatch: 281 | Cost:    133629.031
Minibatch: 301 | Cost:    133342.750
Minibatch: 321 | Cost:    132918.531
Minibatch: 341 | Cost:    132692.984
Minibatch: 361 | Cost:    132414.656
Minibatch: 381 | Cost:    132018.453
Minibatch: 401 | Cost:    131790.484
Minibatch: 421 | Cost:    131469.344
Minibatch: 441 | Cost:    131151.281
Minibatch: 461 | Cost:    130808.078
Minibatch: 481 | Cost:    130494.188
Epoch:     871 | AvgCost: 5432653.081
Minibatch: 001 | Cost:    3840.594
Minibatch: 021 | Cost:    12404.703
Minibatch: 041 | Cost:    12974.740
Minibatch: 061 | Cost:    14573.736
Minibatch: 081 | Cost:    16081.081
Minibatch: 101 | Cost:    13976.771
Minibatch: 121 | Cost:    13744.648
Minibatch: 141 | Cost:    13637.495
Minibatch: 161 | Cost:    13359.167
Minibatch: 181 | Cost:    13314.227
Minibatch: 201 | Cost:    14072.133
Minibatch: 221 | Cost:    13780.275
Minibatch: 241 | Cost:    12887.495
Minibatch: 261

Minibatch: 181 | Cost:    4818.704
Minibatch: 201 | Cost:    4909.342
Minibatch: 221 | Cost:    4969.697
Minibatch: 241 | Cost:    4862.866
Minibatch: 261 | Cost:    4832.895
Minibatch: 281 | Cost:    5210.127
Minibatch: 301 | Cost:    5254.269
Minibatch: 321 | Cost:    5093.864
Minibatch: 341 | Cost:    8673.462
Minibatch: 361 | Cost:    7200.111
Minibatch: 381 | Cost:    6784.813
Minibatch: 401 | Cost:    8823.633
Minibatch: 421 | Cost:    7105.374
Minibatch: 441 | Cost:    5140.530
Minibatch: 461 | Cost:    4887.142
Minibatch: 481 | Cost:    4877.980
Epoch:     880 | AvgCost: 3809837.340
Minibatch: 001 | Cost:    144309.281
Minibatch: 021 | Cost:    167866.094
Minibatch: 041 | Cost:    167345.938
Minibatch: 061 | Cost:    166962.266
Minibatch: 081 | Cost:    166774.094
Minibatch: 101 | Cost:    166628.219
Minibatch: 121 | Cost:    168863.875
Minibatch: 141 | Cost:    166412.812
Minibatch: 161 | Cost:    166414.328
Minibatch: 181 | Cost:    166240.016
Minibatch: 201 | Cost:    166406

Minibatch: 061 | Cost:    4760.255
Minibatch: 081 | Cost:    4720.701
Minibatch: 101 | Cost:    4697.846
Minibatch: 121 | Cost:    4689.341
Minibatch: 141 | Cost:    4691.949
Minibatch: 161 | Cost:    4702.329
Minibatch: 181 | Cost:    4716.475
Minibatch: 201 | Cost:    4731.078
Minibatch: 221 | Cost:    4745.724
Minibatch: 241 | Cost:    4750.347
Minibatch: 261 | Cost:    4826.809
Minibatch: 281 | Cost:    4855.374
Minibatch: 301 | Cost:    4912.325
Minibatch: 321 | Cost:    4950.102
Minibatch: 341 | Cost:    4947.772
Minibatch: 361 | Cost:    4935.476
Minibatch: 381 | Cost:    4931.638
Minibatch: 401 | Cost:    4933.565
Minibatch: 421 | Cost:    4938.243
Minibatch: 441 | Cost:    4943.559
Minibatch: 461 | Cost:    4948.132
Minibatch: 481 | Cost:    4951.254
Epoch:     889 | AvgCost: 4229652.147
Minibatch: 001 | Cost:    122287.102
Minibatch: 021 | Cost:    90896.828
Minibatch: 041 | Cost:    91075.633
Minibatch: 061 | Cost:    91134.516
Minibatch: 081 | Cost:    91162.734
Minibatch: 

Minibatch: 461 | Cost:    89240.023
Minibatch: 481 | Cost:    89202.234
Epoch:     897 | AvgCost: 5702964.366
Minibatch: 001 | Cost:    8457.041
Minibatch: 021 | Cost:    7090.659
Minibatch: 041 | Cost:    7431.793
Minibatch: 061 | Cost:    7228.925
Minibatch: 081 | Cost:    7793.578
Minibatch: 101 | Cost:    7054.825
Minibatch: 121 | Cost:    7309.388
Minibatch: 141 | Cost:    7109.725
Minibatch: 161 | Cost:    6942.440
Minibatch: 181 | Cost:    7130.326
Minibatch: 201 | Cost:    7039.407
Minibatch: 221 | Cost:    6923.569
Minibatch: 241 | Cost:    6981.738
Minibatch: 261 | Cost:    6984.935
Minibatch: 281 | Cost:    6916.495
Minibatch: 301 | Cost:    6918.603
Minibatch: 321 | Cost:    6929.006
Minibatch: 341 | Cost:    6901.543
Minibatch: 361 | Cost:    6890.282
Minibatch: 381 | Cost:    6890.992
Minibatch: 401 | Cost:    6881.396
Minibatch: 421 | Cost:    6871.095
Minibatch: 441 | Cost:    6865.335
Minibatch: 461 | Cost:    6860.628
Minibatch: 481 | Cost:    6851.638
Epoch:     898 

In [ ]:
len(cost_history)

In [ ]:
plt.plot(np.arange(25000), cost_history)
plt.show()